In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, util

In [5]:
base_prompts_1 = 'Why do humans need sleep?'
base_prompts_2 = 'How does photosynthesis work in plants?'
base_prompts_3 = 'Describe happiness without using the word "happy".'

In [15]:
variant_prompts_1 = [
    'What makes sleep essential for humans?',
    'How does sleep benefit the human body and mind?',
    'What role does sleep play in human health and functioning?',
    'Why is it necessary for people to sleep?',
    'In what ways is sleep crucial to human well-being?',
    "What are the reasons humans can't function without sleep?",
    "Why is getting enough sleep important for humans?",
    "What happens to the human body and brain that makes sleep a necessity?"
]

In [16]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [17]:
base_p1_emb = model.encode(base_prompts_1)

In [18]:
variants_p1_emb = model.encode(varient_prompts_1)

In [19]:
similarities = util.cos_sim(base_p1_emb, varients_p1_emb).cpu().numpy().flatten()
print(b_v_p1_sim)

tensor([[0.8500, 0.7480, 0.6935, 0.9188, 0.7535, 0.8178, 0.8823, 0.8255]])


In [27]:
def filter_v_sim(base,variant,model, min_sim=0.85):
    base_emb = model.encode(base, convert_to_tensor=True)
    var_embs = model.encode(variant, convert_to_tensor=True)
    sims = util.cos_sim(base_emb, var_embs).cpu().numpy().flatten()

    df = pd.DataFrame({
        "variant": variant,
        "similarity": sims,
        "keep": [(min_sim <= s) for s in sims]
    })
    return df.sort_values("similarity", ascending=False)

    filtered_df = filter_variants_by_similarity(
    base_prompt_1, variant_prompts_1, model, min_sim=0.85)

    print(filtered_df)

In [28]:
filter_v_sim(base_prompts_1,variant_prompts_1,model)

,variant,similarity,keep
3,Why is it necessary for people to sleep?,0.918821,True
6,Why is getting enough sleep important for humans?,0.882275,True
0,What makes sleep essential for humans?,0.850024,True
7,What happens to the human body and brain that ...,0.825465,False
5,What are the reasons humans can't function wit...,0.817788,False
4,In what ways is sleep crucial to human well-be...,0.753535,False
1,How does sleep benefit the human body and mind?,0.747996,False
2,What role does sleep play in human health and ...,0.693460,False
